# Method 2

Extracting plane polygon based on the segmented roof plane point clouds, then organize them as 3D roof structures

### Imported libraries

In [1]:
import os
import numpy as np
import laspy as lp
import open3d as o3d
import geopandas as gpd
import laspy as lp
import matplotlib.pyplot as plt
from shapely.geometry import Point


### Data 


In [2]:
pointcloud_data = "sample_roofdata_50"
footprint_data = "cropped_data/clipped_final_area.shp"

### 3D models 

In [ ]:
for roof in os.listdir(pointcloud_data):
    if roof.endswith(".laz"):
        file_path = os.path.join(pointcloud_data, roof)
        
        with lp.open(file_path, laz_backend=lp.LazBackend) as fh:
            las_data = fh.read()
            x = las_data.x
            y = las_data.y
            z = las_data.z
            r = las_data.red
            g = las_data.green
            b = las_data.blue
            

            #Normalize RGB values to [0,1]
            if np.max(r) > 255:  # If 16-bit, convert to 8-bit
                r = r / 65535
                g = g / 65535
                b = b / 65535
            else:  # If already 8-bit
                r = r / 255
                g = g / 255
                b = b / 255

            # Create RGB color array in shape (N,3)
            colors = np.vstack((r, g, b)).T  
            points = np.vstack((las_data.x, las_data.y, las_data.z)).T  #(N,3) array

            #planar patch detection
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(points)
            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))
            assert (pcd.has_normals())
            oboxes = pcd.detect_planar_patches(
                normal_variance_threshold_deg=15,
                coplanarity_deg=50,
                outlier_ratio=5,
                min_plane_edge_length=3,
                min_num_points=0,
                search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50)
                )

            print("Detected {} patches".format(len(oboxes)))

            geometries = []
            for obox in oboxes:
                mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox)
                mesh.paint_uniform_color(obox.color)
                geometries.append(mesh)
                geometries.append(obox)
            geometries.append(pcd)
            
            o3d.visualization.draw_geometries(geometries,
                                              zoom=0.62,
                                  )
            


Detected 2 patches
Detected 2 patches


2025-03-15 14:05:33.986 Python[25767:726207] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-15 14:05:33.986 Python[25767:726207] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Detected 2 patches
Detected 5 patches
Detected 3 patches
Detected 2 patches
Detected 4 patches
Detected 6 patches
Detected 7 patches
Detected 6 patches
Detected 2 patches
Detected 4 patches


In [3]:
footprint = gpd.read_file(footprint_data)
footprint = footprint.to_crs(25832)

#Get corners from buildings
def extract_corners(polygon):
    if polygon.geom_type == "Polygon":
        return [Point(x, y) for x, y in polygon.exterior.coords] 
    elif polygon.geom_type == "MultiPolygon":
        return [Point(x, y) for poly in polygon.geoms for x, y in poly.exterior.coords] 
    else:
        return None  

footprint["corners"] = footprint["geometry"].apply(extract_corners)

footprint = footprint.to_crs(25832)
footprint.head()

,osm_id,code,fclass,name,type,geometry,corners
0,5079678,1500,building,Trondheim rådhus,public,"POLYGON ((569712.622 7034071.974, 569716.694 7...","[POINT (569712.6216782562 7034071.974002578), ..."
1,17693566,1500,building,Donjon,None,"POLYGON ((570407.846 7033920.271, 570393.136 7...","[POINT (570407.8464293351 7033920.2707800865),..."
2,19145582,1500,building,Trondheim døvekirke,church,"POLYGON ((569828.869 7032858.683, 569829.349 7...","[POINT (569828.8686698809 7032858.682878285), ..."
3,22811509,1500,building,Byåsen butikksenter,retail,"POLYGON ((567499.029 7032887.369, 567499.389 7...","[POINT (567499.0288805708 7032887.368571803), ..."
4,22915083,1500,building,Bunnpris & Gourmet Tyholt,retail,"POLYGON ((571438.583 7033094.082, 571441.699 7...","[POINT (571438.5825759375 7033094.081795346), ..."


In [ ]:

pointcloud_data = "sample_roofdata_50"
roof_geometries = []

for roof in os.listdir(pointcloud_data):
    if roof.endswith(".laz"):
        file_path = os.path.join(pointcloud_data, roof)
        
        with lp.open(file_path, laz_backend=lp.LazBackend) as fh:
            las_data = fh.read()
            x = las_data.x
            y = las_data.y
            z = las_data.z
            r = las_data.red
            g = las_data.green
            b = las_data.blue
            

            # 🎨 Normalize RGB values to [0,1]
            if np.max(r) > 255:  # If 16-bit, convert to 8-bit
                r = r / 65535
                g = g / 65535
                b = b / 65535
            else:  # If already 8-bit
                r = r / 255
                g = g / 255
                b = b / 255

            # Create RGB color array in shape (N,3)
            colors = np.vstack((r, g, b)).T  

            points = np.vstack((las_data.x, las_data.y, las_data.z)).T  # (N,3) array


            pcd = o3d.geometry.PointCloud()
            
            pcd.points = o3d.utility.Vector3dVector(points)
            pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))
            assert (pcd.has_normals())
            oboxes = pcd.detect_planar_patches(
            
            normal_variance_threshold_deg=15,
            
            coplanarity_deg=50,
            
            outlier_ratio=5,
            
            min_plane_edge_length=3,
            
            min_num_points=0,
            
            search_param=o3d.geometry.KDTreeSearchParamKNN(knn=50))

            print("Detected {} patches".format(len(oboxes)))

            geometries = []
            for obox in oboxes:
                mesh = o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(obox)
                mesh.paint_uniform_color(obox.color)
                geometries.append(mesh)
                geometries.append(obox)
                center = obox.get_center()
            geometries.append(pcd)
            center_point = Point(float(center[0]), float(center[1])) 
                 
            
            o3d.visualization.draw_geometries(geometries,
                                              zoom=0.62,
                                  )
            roof_geometries.append({
                'roof_id': roof,  
                'x': x,
                'y': y,
                'z': z,
                'NumberOfCorners': geometries[0],
                'center_point' : center_point,
                'Polygon': geometries,
                

            })

            gdf_roofs = gpd.GeoDataFrame(roof_geometries)
            

gdf_roofs = gdf_roofs.set_geometry("center_point")
gdf_roofs = gdf_roofs.set_crs(25832)
gdf_roofs.head()

        

Detected 2 patches


In [1]:
# dataframe where buildings is connected with the nearest roof
footprint = footprint.to_crs(25832)
gdf_roofs = gdf_roofs.to_crs(25832)

combined_footprint_pointcloud: gpd.GeoDataFrame = footprint.sjoin_nearest(gdf_roofs, how="inner", distance_col="distance", max_distance=5)
combined_footprint_pointcloud.head()



NameError: name 'footprint' is not defined